# __Auto-ML을 사용하여 예측 모델 만들기__

**회귀 작업 이해하기** 
--- 
👉 회귀 작업은 목표값(Target)이 연속성을 지닌 숫자를 예측하기 위해 사용하는 머신러닝(기계학습/Machine Learning)의 한 형태입니다. 예를 들어, 기상 데이터가 주어졌을 때 내일 기온을 예측하거나, 특정지역의 집값을 예측할 때 사용할 수 있습니다.


**본 튜토리얼에서는**
--- 
👉 대표적인 머신러닝 경진대회 플랫폼인 캐글의 입문자를 위한 Bike Sharing Demand 데이터 세트를 사용하여 자전거 수요 예측 회귀 모델을 만듭니다. 이 대회의 목표는 아래와 같습니다. (참고로, 해당 대회의 데이터는 2011년부터 2012년까지 날짜와 시간, 기온, 습도, 풍속 등의 정보를 기반으로 자전거 대여 횟수가 기재되어 있습니다.)

- 특정 날짜에서 시간대별 자전거 대여 수량 예측하기

## __0. 데이터 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/quick_start/how_to_use_ThanoSQL/#5-thanosql)
에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.   

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은 개인 토큰>

In [2]:
%%thanosql
COPY bike_sharing_train 
OPTIONS(overwrite= True)
FROM "tutorial_data/bike_sharing_data/bike_sharing_train.csv"

Success


In [3]:
%%thanosql
COPY bike_sharing_test 
OPTIONS(overwrite = True)
FROM "tutorial_data/bike_sharing_data/bike_sharing_test.csv"

Success
CPU times: user 38.7 ms, sys: 15.5 ms, total: 54.2 ms
Wall time: 543 ms


__OPTIONS__ : 

__overwrite가 True일 때__, 사용자는 이전 생성했던 데이터 테이블과 같은 이름의 데이터 테이블을 생성할 수 있습니다.  
반면, __overwrite가 False일 때__, 사용자는 이전에 생성했던 데이터 테이블과 같은 이름의 데이터 테이블을 생성할 수 없습니다.

## __1. 데이터 세트 확인__

본 튜토리얼을 진행하기 위해 우리는 ThanoSQL DB에 저장되어 있는 bike_sharing_train 테이블을 사용합니다. 아래의 쿼리문을 실행하여 테이블 내용을 확인합니다.

In [4]:
%%thanosql
SELECT * 
FROM bike_sharing_train 
LIMIT 5

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0,1


__데이터 이해하기__ 

 __bike_sharing_train__ 데이터 세트에는 2011년 1월부터 2012년 12월까지 날짜와 시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수에 대한 정보를 담고 있습니다.  
-  datetime : 시간별 날짜  
-  season : 계절(1 = 봄, 2 = 여름, 3 = 가을, 4 = 겨울)  
-  holiday : 휴일(0 = 휴일이 아닌 날, 1 = 주말을 제외한 국경일 등의 휴일)  
-  workingday   : 작업일(0 = 주말 및 휴일, 1 = 주말 및 휴일이 아닌 주중)  
-  weather   : 날씨  
-  temp   : 온도  
-  atemp   : 체감온도  
-  humidity   : 상대습도  
-  windspeed   : 풍속  
-  count   : 대여 횟수

## __2. 회귀 모델 생성__

이전 단계에서 확인한 bike_sharing_train 데이터 세트를 사용하여 자전거 수요 예측 회귀 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 bike_regression이라는 이름의 모델을 만듭니다.  

(쿼리 실행 시 예상 소요 시간 : 8 min)

In [5]:
%%thanosql
BUILD MODEL tutorial_automl_regressor
USING AutomlRegressor
OPTIONS (
    target='count', 
    impute_type='simple', 
    datetime_attribs=['datetime'],
    time_left_for_this_task = 300,
    overwrite = True
    ) 
AS
SELECT *
FROM bike_sharing_train

Success


- "__BUILD MODEL__" 쿼리 구문을 사용하여 bike_regression라는 모델을 만들고 학습시킵니다. 
- "__OPTIONS__"의 "target"에는 회귀 예측 모델의 목표값이 되는 열의 이름을 적어줍니다. "impute_type"의 경우에는 데이터 세트의 빈 값에 대한 처리를 의미합니다. "datetime_attribs"에는 날짜 형식의 데이터를 적어주면 머신러닝 모델 생성을 진행할 수 있습니다.

__overwrite가 True일 때__, 사용자는 이전 생성했던 데이터 테이블과 같은 이름의 데이터 테이블을 생성할 수 있습니다.  
반면, __overwrite가 False일 때__, 사용자는 이전에 생성했던 데이터 테이블과 같은 이름의 데이터 테이블을 생성할 수 없습니다.

## __3. 생성된 모델 평가__

아래의 쿼리문을 실행하여 이전 단계에서 만든 예측 모델의 성능을 평가합니다.

In [6]:
%%thanosql
EVALUATE USING tutorial_automl_regressor 
OPTIONS (
    target='count'
    ) 
AS
SELECT *
FROM bike_sharing_train

,metrics,value
0,MAE,70.1031
1,MSE,9679.4456
2,R2,0.3523
3,RMSLE,1.1241
4,MAPE,0.4526


__쿼리 세부 정보__ 
- "__EVALUATE USING__" 쿼리 구문을 사용하여 구축한 bike_regression 모델을 평가합니다. 
- "__OPTIONS__"의 "target"에는 회귀 예측 모델의 목표값이 되는 열(Column)의 이름(count)을 적어줍니다.

## __4. 생성된 모델을 사용하여 자전거 대여 수량 예측__

이전 단계에서 만든 수요 예측 모델로 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, bike_shaing_test)에 있는 10개의 데이터에 대한 자전거 대여 수량을 예측해 봅니다.

In [7]:
%%thanosql
PREDICT USING tutorial_automl_regressor 
AS
SELECT *
FROM bike_sharing_test
LIMIT 10

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,predicted
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,133.351361
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,83.304852
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,83.304852
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,89.718215
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,89.718215
5,2011-01-20 05:00:00,1,0,1,1,9.84,11.365,60,15.0013,104.283055
6,2011-01-20 06:00:00,1,0,1,1,9.02,10.605,60,15.0013,100.676295
7,2011-01-20 07:00:00,1,0,1,1,9.02,10.605,55,15.0013,91.813349
8,2011-01-20 08:00:00,1,0,1,1,9.02,10.605,55,19.0012,99.065202
9,2011-01-20 09:00:00,1,0,1,2,9.84,11.365,52,15.0013,108.508070


__쿼리 세부 정보__

- "__PREDICT USING__" 쿼리 구문을 사용하여 <mark style="background-color:#E9D7FD ">bike_regression</mark> 모델을 예측에 사용합니다. 
- "__PREDICT__"의 경우 생성된 모델의 절차를 따르기 때문에 특별한 처리가 필요없습니다.